In [26]:
import pickle

# xTrain.pkl 파일 로드
with open(r'C:\Users\bluecom011\Desktop\module_2\wine_quality\xTrain.pkl', 'rb') as file:
    x = pickle.load(file)

# yTrain.pkl 파일 로드
with open(r'C:\Users\bluecom011\Desktop\module_2\wine_quality\yTrain.pkl', 'rb') as file:
    y = pickle.load(file)

# 데이터 형태 확인
x.shape, y.shape


((1599, 11), (1599,))

In [27]:
from sklearn.model_selection import train_test_split

# 훈련 세트와 테스트 세트로 분할
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

x_train.shape, x_test.shape, y_train.shape, y_test.shape

((1279, 11), (320, 11), (1279,), (320,))

In [32]:
import torch
import numpy as np
from torch.utils.data import TensorDataset, DataLoader

# 데이터를 PyTorch 텐서로 변환
x_train_tensor = torch.tensor(np.tile(x_train, (1, 3))[:, :30], dtype=torch.float32)
x_test_tensor = torch.tensor(np.tile(x_test, (1, 3))[:, :30], dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# 타일 확장을 통해 형성된 텐서 데이터를 TensorDataset에 포장
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)

# DataLoader 설정
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

# 이후 데이터 로더를 사용하여 모델 학습 및 평가를 진행할 수 있음


In [29]:
import torch
import torchvision.transforms as transforms
from torchvision.models import resnet18
from torch.utils.data import DataLoader, TensorDataset

# x가 원본 데이터라고 가정, X_img_like는 이미 3x10 형태로 변환된 상태
X_img_like = X_expanded.reshape(-1, 3, 10)  # 예시 코드에서 사용된 변환

# 데이터를 32x32 형태로 변환하기 위한 임시 방안: Zero-padding을 사용
# 이를 위해 먼저 30x30 형태로 타일 확장 후, 각각의 샘플 주위에 zero-padding을 적용
X_padded = np.pad(X_img_like.repeat(10, axis=1), ((0,0), (1,1), (0,22)), 'constant', constant_values=0)

# PyTorch 텐서로 변환 (이미 3채널 데이터라고 가정)
X_tensor = torch.tensor(X_padded, dtype=torch.float32).unsqueeze(1) # 채널 차원 추가

# 채널 차원을 RGB로 복제하기
X_tensor_rgb = X_tensor.repeat(1, 3, 1, 1)

# 데이터셋과 데이터로더 준비
dataset = TensorDataset(X_tensor_rgb)
dataloader = DataLoader(dataset, batch_size=32)

# ResNet18 모델 로드 및 평가 모드 설정
model = resnet18(pretrained=True)
model.eval()

# 모델을 통해 데이터 전달
for inputs in dataloader:
    outputs = model(inputs[0])  # 수정: 채널 수 조정이 이미 되어 있으므로 추가 조정 필요 없음
    # 출력 처리 ...


In [34]:
# 올바른 타일 반복을 위한 수정
X_img_expanded_correct = np.tile(X_img_like, (10, 1, 1))

# 최종적으로 얻고자 하는 형태로 재구성
X_final_correct = X_img_expanded_correct.reshape(-1, 30, 10)

# 결과 확인
print(X_final_correct.shape)  # 출력 예: (1599, 30, 10), 각 샘플이 30행 10열 형태의 '이미지'로 변환됨


(1599, 30, 10)


In [11]:
import numpy as np

# x가 원본 데이터라고 가정합니다 (예: x_train)
# 각 샘플을 1행 30열로 확장
X_expanded = np.tile(x, (1, 3))[:, :30]  # 11개 특성을 3배 확장한 후 30열로 자름

# 확장된 데이터를 2차원 형태로 변환 (예: 3x10 형태로 재구성)
# 이를 위해 각 샘플을 3행 10열의 2차원 배열로 재구성
X_img_like = X_expanded.reshape(-1, 3, 10)  # 여기서 -1은 자동으로 계산된 샘플 수를 의미

# 결과 확인
print(X_img_like.shape)  # 출력 예: (1599, 3, 10), 각 샘플이 3x10 형태의 '이미지'로 변환됨


(1599, 3, 10)


In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.models import resnet18
from torch.utils.data import DataLoader, random_split, TensorDataset
from sklearn.model_selection import train_test_split

# 사전 훈련된 ResNet18 모델 로드
base_model = resnet18(pretrained=True)

# 마지막 층 교체 (이진 분류)
num_features = base_model.fc.in_features
base_model.fc = nn.Sequential(
    nn.Linear(num_features, 1),
    nn.Sigmoid()
)

# 옵티마이저 및 손실 함수 설정
optimizer = optim.Adam(base_model.parameters(), lr=0.001)
criterion = nn.BCELoss()

# 데이터 로더 설정
transformed_images_rgb = ...  # 이전 단계에서 데이터를 준비하는 코드
y_train_tensor = torch.tensor(y_train, dtype=torch.long)

dataset = TensorDataset(transformed_images_rgb, y_train_tensor)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# 모델 학습 함수 정의
def train_model(model, criterion, optimizer, train_loader, val_loader, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels.unsqueeze(1).float())
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for inputs, labels in val_loader:
                outputs = model(inputs)
                loss = criterion(outputs, labels.unsqueeze(1).float())
                val_loss += loss.item()

        print(f'Epoch {epoch+1}/{num_epochs}, Train Loss: {running_loss/len(train_loader)}, Val Loss: {val_loss/len(val_loader)}')

# 모델 학습
train_model(base_model, criterion, optimizer, train_loader, val_loader)


c:\Users\bluecom011\miniconda3\envs\torch\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\bluecom011\miniconda3\envs\torch\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


AttributeError: 'ellipsis' object has no attribute 'size'